In [6]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from PIL import Image
import json

import segmap_utils, dataloader

In [2]:
df = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz3d_and_gz_desi_matches.csv')[:100]

In [3]:

df['local_desi_jpg_loc'] = 'data/desi/jpg/' + str(df['brickid']) + '/' + str(df['brickid']) + '_' + str(df['objid']) + '.jpg'

In [7]:
df['segmap_json_loc'] = df['local_gz3d_fits_loc'].str.replace('/fits_gz/', '/segmaps/', regex=False).str.replace('.fits.gz', '.json', regex=False)
galaxy = df.iloc[32]
with open(galaxy['segmap_json_loc'], 'r') as f:
    segmaps = json.load(f)
spiral_marks = segmaps['spiral']
_, fig, ax = dataloader.construct_segmap_image(galaxy, spiral_marks, True)

KeyError: 'ra_subject'

In [ ]:
im = fits.open(galaxy['local_desi_fits_loc'])[0].data
im = im.transpose(1, 2, 0)  # channels last

# def crop_in_third(img):
#     width = img.shape[0]//3
#     print(img.shape, width)
#     return crop_center(img, width, width)

# crop DESI to have GZ3D FoV
# 52 arcsec at 0.262 arcsec/pixel
# 52 / 0.262 = 198

def crop_center(img,cropx=198,cropy=198):
    y , x, _ = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

im = crop_center(im)
im = im.transpose(2, 0, 1)  # channels first
# print(im)

In [ ]:

desi_im = utils.fits_to_png(im).transpose(Image.FLIP_TOP_BOTTOM).resize((525, 525))
plt.imshow(desi_im)
# im.min(), im.max()

"GZ:3D images were selected to have a pixel scale of 0.099 arcsec per pixel
and were generated to have a ﬁeld of view of 52 arcsec (525 × 525
pixels), selected to be twice the size of the largest MaNGA bundle
on the sky."

In [ ]:
manga_im = fits.open(galaxy['local_gz3d_fits_loc'])[0].data
plt.imshow(manga_im)

In [ ]:
seg_im = fits.open(galaxy['local_gz3d_fits_loc'])[3].data
plt.imshow(seg_im)

In [ ]:
seg_im.shape, desi_im.size

In [ ]:
plt.imshow(desi_im)
plt.imshow(seg_im, alpha=.1)
plt.axis('off')

In [ ]:
"""
The image stacks use a simple tangent-plane (WCS TAN) projection around the brick center.
The projections for the griz filters are identical, 
with a pixel scale of 0.262″/pix. 
"""

In [ ]:
# TODO need a flip